# Hive assignment 1

The purpose of this task is to create an external table on the posts data of the `stackoverflow.com` website.

## Step 4. Complete the assignment

In [20]:
%%writefile task1_create_external_table.hql
-- Create external table posts_sample_external with suitable values
-- Your code here

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

USE demodb;
DROP TABLE IF EXISTS posts_sample_external;

Create EXTERNAL TABLE posts_sample_external (
                id STRING,
                year STRING,
                month STRING
)

ROW FORMAT
    SERDE 'org.apache.hadoop.hive.contrib.serde2.RegexSerDe'
    WITH SERDEPROPERTIES (
    "input.regex" = '.*?(?=.*\\bId=\"(\\d+)\").*(?<=\\bCreationDate\\b=\")(\\d*)-(\\d*).*'
)
LOCATION '/data/stackexchange1000/posts'
TBLPROPERTIES ("skip.header.line.count"="1");

Overwriting task1_create_external_table.hql


Make sure that you have created your table correctly. Select the first 10 posts from the dataset.

In [21]:
%%writefile task1_check_select.hql

-- Write select query for the first 10 rows
-- Your code here

USE demodb;
SELECT *
FROM posts_sample_external
LIMIT 10;

Overwriting task1_check_select.hql


Create managed table `posts_sample`. Create the partition by the month and by the year. 

In [22]:
%%writefile task1_create_managed_table.hql
-- create managed table
-- Check that this table contains info about year and month
-- Your code here

USE demodb;
DROP TABLE IF EXISTS posts_sample;

CREATE TABLE posts_sample (
             id STRING
) 
PARTITIONED BY (year STRING, month STRING);

Overwriting task1_create_managed_table.hql


In [23]:
%%writefile task1_create_managed_table_check.hql
-- Check that this table contains info about year and month

USE demodb;
DESCRIBE posts_sample

Overwriting task1_create_managed_table_check.hql


Populate data from the table `posts_sample_external` to the table `posts_sample`. Don't forget about the partitioning rules!

In [24]:
%%writefile task1_insert_table.hql

-- Insert data to the managed table

set hive.exec.max.dynamic.partitions=300;
set hive.exec.max.dynamic.partitions.pernode=256;

USE demodb;
-- filling managed posts table from external one
SET hive.exec.dynamic.partition.mode=nonstrict;

-- Your code here for inserting data
FROM posts_sample_external
INSERT OVERWRITE TABLE posts_sample
PARTITION (year, month)
SELECT id, year, month
WHERE year IS NOT NULL and month is not NULL;

Overwriting task1_insert_table.hql


Make sure that your table contains appropriate data about posts

In [25]:
%%writefile task1_watch_new_table.hql
-- Your code here
USE demodb;
SELECT * FROM posts_sample LIMIT 10;

Overwriting task1_watch_new_table.hql


Take the third row of the dataset in the ascending order for the posts (firstly by year, after that by month)

In [26]:
%%writefile task1_result.hql
-- Your code here

USE demodb;

SELECT concat_ws("\t", year, concat_ws("-", year, month), c)
FROM (
    SELECT year, month, STRING(COUNT(*)) as c, ROW_NUMBER() OVER (ORDER BY year, month) as row
    FROM posts_sample
    GROUP BY year, month
) as tmp
WHERE row=3;

Overwriting task1_result.hql


## Step 5. Submission part. Do not touch!! And simple run all cells below!

Copy your notebook from the steps <a href="#Step-4.-Complete-the-assignment">Step 4</a> and <a href="#Step-5.-Submission-part.-Do-not-touch!!-And-simple-run-all-cells-below!">Step 5</a> to the new notebook. Run all the cells! And submit the copied notebook!

In [27]:
!cat task1_create_external_table.hql > task1.hql
!cat task1_create_managed_table.hql >> task1.hql
!cat task1_insert_table.hql >> task1.hql
!cat task1_result.hql >> task1.hql

Take a look at your submission query!

In [28]:
!cat task1.hql

-- Create external table posts_sample_external with suitable values
-- Your code here

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

USE demodb;
DROP TABLE IF EXISTS posts_sample_external;

Create EXTERNAL TABLE posts_sample_external (
                id STRING,
                year STRING,
                month STRING
)

ROW FORMAT
    SERDE 'org.apache.hadoop.hive.contrib.serde2.RegexSerDe'
    WITH SERDEPROPERTIES (
    "input.regex" = '.*?(?=.*\\bId=\"(\\d+)\").*(?<=\\bCreationDate\\b=\")(\\d*)-(\\d*).*'
)
LOCATION '/data/stackexchange1000/posts'
TBLPROPERTIES ("skip.header.line.count"="1");-- create managed table
-- Check that this table contains info about year and month
-- Your code here

USE demodb;
DROP TABLE IF EXISTS posts_sample;

CREATE TABLE posts_sample (
             id STRING
) 
PARTITIONED BY (year STRING, month STRING);
-- Insert data to the managed table

set hive.exec.max.dynamic.par

In [29]:
%%javascript

$(document).ready(function() {
    console.log('Ready');
    
    
    function is_hive_command(list_tokens) {
        return list_tokens.indexOf('hive') > -1 && 
             list_tokens.indexOf('f') > -1 &&
             list_tokens.indexOf('-') > -1 && 
             list_tokens.indexOf('!') > -1 &&
             list_tokens.indexOf('hql') > -1 && 
             list_tokens.indexOf('writefile') == -1;
    } 
    
    function collectText(input_tag) {

        var result_string = [];
        $.each($(input_tag).children(), function(index, child) {
            result_string.push($(child).text());
        });
        return [result_string, is_hive_command(result_string)];
    };
    
    var filtered_results = $(".cell.code_cell.rendered").filter(function(index, element) {
        var out = collectText($(element).find('.CodeMirror-line').find('span'));
        console.log(out);
        return collectText($(element).find('.CodeMirror-line').find('span'))[1];
    });
    $(filtered_results).remove();
});

<IPython.core.display.Javascript object>

In [30]:
%%bash
hive -f task1.hql

2008	2008-10	73



Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar]
OK
Time taken: 1.043 seconds
OK
Time taken: 1.73 seconds
OK
Time taken: 0.602 seconds
OK
Time taken: 0.021 seconds
OK
Time taken: 3.488 seconds
OK
Time taken: 0.365 seconds
OK
Time taken: 0.018 seconds
Query ID = jovyan_20181113122626_c058db4a-43de-469c-b7c4-a112ad1584d1
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1542041219823_0014, Tracking URL = http://c5bd5fec03a1:8088/proxy/application_1542041219823_0014/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1542041219823

Congratulations! You have completed the assignment! Now you can submit it to the system and get your results!

Copy your notebook from the steps <a href="#Step-4.-Complete-the-assignment">Step 4</a> and <a href="#Step-5.-Submission-part.-Do-not-touch!!-And-simple-run-all-cells-below!">Step 5</a> to the new notebook. Run all the cells! And submit the copied notebook!